In [1]:
import QuantLib as ql
import pandas as pd
import numpy as np
import os 
import quantFin as qf
import pymongo as py
import mysql.connector
import scipy.optimize as sp
import math
import matplotlib.pyplot as plt
import time
import datetime as dt
import sys

In [168]:
config = {
    'user': 'root',
    'password': 'cba123',
    'host': '34.66.65.69',
    'database':'ACT_01',
    'use_pure':True,

}
cnxn = mysql.connector.connect(**config)
cursor = cnxn.cursor()

In [169]:
#get mortality assumptions and store them in a dictionary for quick referencing
cursor.execute("select * from mortality_assumptions")
out=cursor.fetchall()
out

wareHouse=qf.AssumptionWareHouse()
for o in out:
    mortality=qf.MortalityAssumption()
    mortality.id=o[0]
    mortality.soa_table_name=o[1]
    mortality.gamma_makeham=o[2]
    mortality.beta_makeham=o[3]
    mortality.alpha_makeham=o[4]
    mortality.gender=o[5]
    wareHouse.mort_dept[mortality.id]=mortality
    

In [167]:
cnxn.close()



In [173]:

cell=qf.Cell()
act1=qf.ActuarialBucket()
cell.mitos.append(act1)
cell.wareHouse.assump_set_dept[1]=qf.AssumptionSet(id=1,mortality_id=1)
cell.wareHouse.assump_set_dept[2]=qf.AssumptionSet(id=2,mortality_id=2)
cell.wareHouse.assump_set_dept[3]=qf.AssumptionSet(id=3,mortality_id=3)
cell.wareHouse.assump_set_dept[4]=qf.AssumptionSet(id=4,mortality_id=4)
cell.wareHouse.assump_set_dept[5]=qf.AssumptionSet(id=5,mortality_id=5)
cell.wareHouse.assump_set_dept[6]=qf.AssumptionSet(id=6,mortality_id=6)
cell.wareHouse.assump_set_dept[7]=qf.AssumptionSet(id=7,mortality_id=7)
cell.wareHouse.assump_set_dept[8]=qf.AssumptionSet(id=8,mortality_id=8)
for o in out:
    mortality=qf.MortalityAssumption()
    mortality.id=o[0]
    mortality.soa_table_name=o[1]
    mortality.gamma_makeham=o[2]
    mortality.beta_makeham=o[3]
    mortality.alpha_makeham=o[4]
    mortality.gender=o[5]
    cell.wareHouse.mort_dept[mortality.id]=mortality

In [174]:
ass_set=qf.AssumptionSet(id=1,mortality_id=1)
act_bucket=qf.ActuarialBucket(id=1,assumption_set_key=1)

wareHouse.assump_set_dept[ass_set.id]=ass_set
cell.mitos.append(act_bucket)
policy=qf.policyHolder()
policy.age=55
policy.assumption_set_id=1



In [201]:
policyX=qf.policyHolder()
policyX.act_assumpset_id=2
policyX.age=55

In [202]:

zz=cell.mitos[0].calculate_qx(policyX,cell.wareHouse)
print(policyX.act_assumpset_id)
print("lx: ",cell.mitos[0].calculate_life_exp(policyX,cell.wareHouse))

startDate=ql.Date(31,10,1929)
aa=qf.create_cashflows(0,startDate,1,zz*1)
print(sum(zz))

n=0
for a in aa:
    
    print(str(policyX.age+n)+':',a.amount())
    n=n+1

2
lx:  29.266393445287697
10.645785755681743
55: 0.003475345243304062
56: 0.0037954599291616714
57: 0.0041513335391272855
58: 0.00454694553633507
59: 0.004986714682475552
60: 0.005475546689208426
61: 0.006018886837684052
62: 0.006622778035810661
63: 0.007293924815283114
64: 0.008039763801761513
65: 0.008868541220773945
66: 0.009789398027415117
67: 0.010812463267739384
68: 0.011948956291419403
69: 0.013211298435550032
70: 0.014613234784421843
71: 0.016169966574665726
72: 0.01789829475312791
73: 0.019816775098542495
74: 0.02194588517805188
75: 0.024308203214486804
76: 0.02692859867613956
77: 0.029834434050939836
78: 0.03305577681161997
79: 0.036625619994255995
80: 0.04058010907216576
81: 0.04495877187899169
82: 0.04980474718308223
83: 0.05516500610005213
84: 0.06109055880805134
85: 0.06763663695498863
86: 0.07486283967248678
87: 0.0828332281946469
88: 0.09161635068571186
89: 0.10128517498988486
90: 0.1119169026349937
91: 0.12359263259496073
92: 0.13639683814828768
93: 0.15041661484692384

In [18]:
cursor.execute("select * from spot_rates where id=3")
out=cursor.fetchall()


In [20]:
bc.valuationDate

Date(27,10,1929)

In [27]:
cfs=[ql.SimpleCashFlow(100,ql.Date(31,10,1931)),ql.SimpleCashFlow(100,ql.Date(31,10,1932))]
spotValues=list(out[0][4:50])
startDate=ql.Date(31,10,1929)
dcurve=qf.create_spotCurve(spotValues,startDate,datetimeStruct2,intRate)


In [22]:
myHandle=ql.YieldTermStructureHandle(dcurve)

In [25]:
ql.CashFlows.npv(cfs,spotCurveHandle,True,ql.Date(31,10,1929))

191.97348605998428

In [151]:
#get mortality assumptions and store them in a dictionary for quick referencing
cursor.execute("select * from qx_data where id=5")
out=cursor.fetchall()
out[0][57]

0.00912

In [ ]:
cfs=[ql.SimpleCashFlow(100,ql.Date(31,10,1931)),ql.SimpleCashFlow(100,ql.Date(31,10,1932))]

result=ql.CashFlows.npv(cfs,discount_curve,True,bc.valuationDate)
result

In [204]:
#get mortality assumptions and store them in a dictionary for quick referencing
cursor.execute("select * from mortality_assumptions where id=2")
xout=cursor.fetchall()
xout

[(2,
  'US_social-Security-2016_F',
  8.762829444019063e-06,
  0.10623560169969236,
  0.0006149490351940046,
  0,
  'US_social-Security-2016_F',
  0,
  'A')]